# Convolutional Autoencoder for Anomaly Detection
## Learning Objectives
1. Learn how to build a Deep Convolutional Autoencoder
2. Learn how to use a trained autoencoder for anomaly detection

In this lab, you will learn how to build an autoencoder model. Autoencoder is a popular unsupervised model which consists of an encoder and a decoder.<br>
An autoencoder is a special type of neural network trained to copy its input to its output. For example, given an image of a handwritten digit, an autoencoder first encodes the image into a lower dimensional latent representation, then decodes the latent representation back to an image. An autoencoder learns to compress the data while minimizing the reconstruction error.

Autoencoders can be used in many ways. In this lab, we will use an autoencoder for anomaly detection, leveraging that an autoencoder will have a harder time reconstructing an anomalous input compared to an input taken from the data distribution it has been trained on."

To learn more about autoencoders, please consider reading chapter 14 from [Deep Learning](https://www.deeplearningbook.org/) by Ian Goodfellow, Yoshua Bengio, and Aaron Courville.


In [ ]:
# Import libraries and modules
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

np.set_printoptions(threshold=np.inf)

### Load MNIST Data

We are going to use the MNIST dataset, which consists of black and white images of handwritten digits. Each image has 28 x 28 pixels and a single channel indicating the level of gray between black and white, yielding a tensor representation with shape(28, 28, 1). Keras comes preloaded with this dataset, which we load in the next cell:"

In [ ]:
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

In [ ]:
plt.imshow(mnist_digits[0], cmap="Greys_r")

## Build Autoencoder model
Let's start building an autoencoder model. 

Autoencoder is a self-supervised model, which uses the same data for input and label. 

![image](https://user-images.githubusercontent.com/6895245/164359926-f72472ca-f2de-4098-bf2b-3e649749f721.png)
It tries to reduce the dimensionality of inputs to a fixed-sized latent space (encoder) and 'reconstruct' the input from the embedding (decoder). So we can regard the encoder part as a dimensionality reduction model like PCA. But whereas PCA learns a linear transformation that projects input into lower-dimensional space, autoencoders learn non-linear transformation using Neural Networks.

### Encoder
Let's build the encoder part at first. Encoder goes from the inputs to the latent space.

![image](https://user-images.githubusercontent.com/6895245/164373070-f7860451-1720-4e4b-aa91-460c4aa98020.png)

**Exercise**: Implement `buil_encoder` function so that it builds and returns a `Sequential` of an encoder with two [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D?version=nightly) layers, [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten) layer and a [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) layer. <br> 
**Hint**: You can use the arguments of the function for Conv2D layers.


In [ ]:
def build_encoder(
    input_shape,
    latent_dim,
    filters_1=32,
    filters_2=64,
    kernel_size_1=3,
    kernel_size_2=3,
    strides_1=2,
    strides_2=2,
):
    encoder = tf.keras.Sequential([])  # TODO
    return encoder

In [ ]:
LATENT_DIM = 2  # for easy visualization
INPUT_SHAPE = (28, 28, 1)  # Size of input data

encoder = build_encoder(INPUT_SHAPE, LATENT_DIM)
encoder.summary()

### Decoder
The decoder goes from the latent space back to the reconstructed image.
In order to reconstruct the original shape (28,28,1), we use [`Conv3DTranspose` layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose) (a.k.a Deconvolution).

![image](https://user-images.githubusercontent.com/6895245/164372979-29797db4-d108-4955-8056-ec668465c3f4.png)

**Exercise**: Complete the `build_decoder` function below so that that it returns a `Sequential` with layer architecture to be the exact reverse of the of the encoder layers. <br>
**Hint**: You can get the decoder layer information from the encoder object: e.g. `encoder.layers[-1].input.shape[1]` will return the input shape of the encoder last layer.

In [ ]:
def build_decoder(latent_dim, encoder):
    decoder = tf.keras.Sequential([])  # TODO
    return decoder

In [ ]:
decoder = build_decoder(LATENT_DIM, encoder)
decoder.summary()

### Construct entire autoencoder

The autoencoder consists of the encoder and the decoder blocks. Let's simply stack these two and build the entire autoencoder model.

In [ ]:
def build_autoencoder(input_shape, latent_dim):
    encoder = build_encoder(input_shape, latent_dim)
    decoder = build_decoder(latent_dim, encoder)
    autoencoder = tf.keras.Sequential([encoder, decoder])
    autoencoder.build(input_shape=(None, *input_shape))
    return encoder, decoder, autoencoder

In [ ]:
encoder, decoder, autoencoder = build_autoencoder(INPUT_SHAPE, LATENT_DIM)

autoencoder.summary()

### Reconstruction Loss

The optimization goal of the autoencoder is to minimize the reconstruction error between input and output. So let's chose MSE as the loss function.

In [ ]:
autoencoder.compile(optimizer=keras.optimizers.Adam(), loss="mse")

## Train autoencoder
Let's run the training by calling `.fit()`.
Here, please notice that both feature and label have the same shape, i.e., (28, 28, 1).

In [ ]:
history = autoencoder.fit(mnist_digits, mnist_digits, epochs=30, batch_size=128)

It looks the reconstruction loss is decreasing gradually.


## Visualize the latent space

We trained an autoencoder model with `LATENT_DIM=2`. Let's visualize the 2 dimensional latent space (output of encoder), and see how it found the pattern from input data.

The function bellow will plot the input images in the latent space by passing them to the encoder. It will also add the indication of the actual image labels (which were not used during the autoencoder training). We observe that the autoencoder was able to group images with the same label into relatively well defined cluster of points in the latent space. In a way, the autoencoder has "guessed" the image labels from the raw images without having the knowledge of the labels during training!


In [ ]:
def plot_label_clusters(encoder, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean = encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    for label in range(10):  # mnist, 10 digits
        cx = np.mean(z_mean[labels == label, 0])
        cy = np.mean(z_mean[labels == label, 1])
        plt.text(
            cx, cy, str(label), color="white", fontsize=25, fontweight="bold"
        )
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()

In [ ]:
(x_train, y_train), _ = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype("float32") / 255
plot_label_clusters(encoder, x_train, y_train)

We can see clusters of digits! That means our encoder part found some patern and embedded information in this 2 dimensional latent space.

## Visualize decoder
Now let's take a look at the decoder part as well.
The helper function below creates grid pairs of two dimensional latent space  (`grid_x` and `grid_y`) in a specified range, then passes them to the decoder and plots the decoder model's output.<br>
It helps us understand how the decoder generate output images from latent spaces.

You can change the `ranges` argument in `[x_min, x_max, y_min, y_max]` format based on the actual range of the clusters in a latent space of the encoder output that is shown above.

In [ ]:
def plot_latent_space(decoder, n=30, ranges=[-1, 1, -1, 1], figsize=15):
    # display a n*n 2D manifold of digits
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(ranges[0], ranges[1], n)
    grid_y = np.linspace(ranges[2], ranges[3], n)[::-1]

    samples = [[x, y] for y in grid_y for x in grid_x]
    x_decoded = decoder.predict(samples)
    figure = np.zeros((digit_size * n, digit_size * n))
    for i in range(n):
        for j in range(n):
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = x_decoded[i * n + j].reshape(digit_size, digit_size)

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()

In [ ]:
plot_latent_space(decoder, n=30, ranges=[-10, 10, -3, 10], figsize=15)

It looks good! Our autoencoder model successfully reconstructs the images from two-dimensional latent space.

Next, let's take a look at how we can use autoencoder to solve the Anomaly Detection problem.

## Anomaly Detection
Now that we demonstrated how autoencoder works, let's utilize this model for anomaly detection.

But why does this work for anomaly detection?

Autoencoder learns the pattern of data by compressing the information and reconstructing it. So if it is successfully trained with 'normal' data, it should be able to rebuild a similar image to 'normal' inputs as we've seen. But if the input was 'anomaly,' it won't reconstruct a similar image since the pattern of anomaly data is very different from normal data.

So in the anomaly case, the error between input and output should be larger than the normal case. We can utilize this characteristic of autoencoder for anomaly detection problems.

### Build AutoEncoder for Anomaly Detection

Before actually using an autoencoder for anomaly detection, let's create a more performant model by specifying a larger latent dimension.

A larger latent dimension means the model can embed richer information in a larger space (like 32).

**Exercise**: In the cell below, use build_autoencoder to instantiate an auto-encoder model. Then write the code to compile and train it.


In [ ]:
encoder, decoder, autoencoder = None  # TODO

In [ ]:
autoencoder.compile()  # TODO
history = autoencoder.fit()  # TODO

### Detect Anomalies

In the next cell, we will detect anomalies by computing the reconstruction error of a given input, and checking if it crosses a threshold, that we establish by computing the mean and the standard deviation of the reconstruction error on the dataset according to the following formula:

`threshold = mean(reconstruction_error) + std(reconstruction_error) + sigma`

where sigma is an hyper-parameter that we need to set. 

In [ ]:
num_sample = 2000  # Number of samples to caliculate threshold

reconstructions = autoencoder.predict(x_train[:num_sample])

train_loss = tf.reduce_mean(
    tf.keras.losses.mae(reconstructions, x_train[:num_sample]), axis=(1, 2)
)

In [ ]:
# Visualize histgram of loss value
plt.hist(train_loss[None, :], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

Let's choose a threshold value that is two standard deviations above the mean.

In [ ]:
# Define Anomaly Threshold
sigma = 2

threshold = 0  # TODO: Define a threshold properly
print("Threshold: ", threshold)

In [ ]:
# Visualize histgram of loss value and anomaly threshold
fig = plt.figure()
plt.hist(train_loss[None, :], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.vlines(threshold, 0, 160, color="red")
plt.show()

### Prediction
At prediction time we

1. reconstruct the input using the autoencoder
1. compute the reconstruction error for that input
1. classify the input as an anomaly if the reconstruction error is above the threshold we have fixed

**Exercise**: In the cell below, define the logic of anomaly detection using threshold.

In [ ]:
# Detection function
def predict(model, data, threshold):
    reconstructions = model(data)
    loss = tf.reduce_mean(
        tf.keras.losses.mae(reconstructions, data), axis=(1, 2)
    )
    return None  # TODO: Define the logic of anomaly detection

This helper function below visualizes input, reconstructed output, and the result of anomaly detection with a comparison of the actual loss value and the threshold.

In [ ]:
# Ploting Utility Function
def plot_reconstruction(model, data, threshold):
    is_anomaly, loss, reconstruction = predict(
        model, tf.expand_dims(data, 0), threshold
    )

    f, axarr = plt.subplots(1, 2)
    lossMSG = f"loss:{loss:.4f}"
    thMSG = f"threshold:{threshold:.4f}"
    text_anomaly = (
        f"Anomaly ({lossMSG} > {thMSG})"
        if is_anomaly
        else f"Normal ({lossMSG} <= {thMSG})"
    )
    title = f.suptitle(text_anomaly)
    c = "r" if is_anomaly else "g"
    plt.setp(title, color=c)

    axarr[0].imshow(data, cmap="Greys_r")
    axarr[0].set_title("Original Image")
    axarr[1].imshow(reconstruction[0], cmap="Greys_r")
    axarr[1].set_title("Reconstructed Image")
    f.show()

### Normal dataset
Let's pass some normal data from the dataset and see how our autoencoder model responds.

In [ ]:
for d in x_train[num_sample - 10 : num_sample]:
    plot_reconstruction(autoencoder, d, threshold)

Looks Like our autoencoder is reconstructing images very well, and the loss value is less than the threshold in most cases.<br>
This is what we expected!

### Try Anomaly Data
Then what happens when it gets 'unusual' data? Let's create some perturbations below and see what happens. We try:
- 90-degree rotated images
- negatively inversed images
- images with white noise

#### 90-degree rotated images

In [ ]:
for d in x_train[num_sample - 10 : num_sample]:
    plot_reconstruction(autoencoder, np.rot90(d), threshold)

It looks a lot of rotated images are detected as anomalies since rotated images are not contained in training and threshold calculation.

But since the `0` value is invariable to angles, rotated `0` can be detected as normal.

Next, let's look at what happens if we create negatives from the original images and provide them to our model.

#### negatively inversed images

In [ ]:
for d in x_train[num_sample - 10 : num_sample]:
    plot_reconstruction(autoencoder, 1 - d, threshold)

They are detected as anomalies with considerable loss value! And we can see our model cannot construct the original image at all.

Then how about the noisy data?

#### images with white noise

In [ ]:
def add_noise_and_scale(d):
    noise = np.random.normal(0, 0.05, size=(28, 28, 1))
    _d = d + noise
    return _d - np.min(_d) / (np.max(_d) - np.min(_d))


for d in x_train[num_sample - 10 : num_sample]:
    plot_reconstruction(autoencoder, add_noise_and_scale(d), threshold)

They are also detected as anomalies!

Interestingly, since our autoencoder was trained with clean images, it tries to 'denoise' and generates clean images as outputs. As a result, the error between the inputs and clean outputs becomes large, and they are detected as anomalies.

Is that mean we can also utilize autoencoders for denoising purposes? Yes, we can do it with [denoising autoencoders](https://www.cs.toronto.edu/~larocheh/publications/icml-2008-denoising-autoencoders.pdf).<br>
But then don't forget to use clean images for training data to make autoencoders capture the pattern of clean data.



## Summary
We learned how to build an autoencoder model by stacking an encoder and a decoder in this lab.<br>
Also, we learned how to utilize an autoencoder for anomaly detection purposes.

# License

Copyright 2022 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.